#DV360 User Audit
Gives DV clients ability to see which users have access to which parts of an account. Loads DV user profile mappings using the API into BigQuery and connects to a DataStudio dashboard.


#License

Copyright 2020 Google LLC,

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

  https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.



#Disclaimer
This is not an officially supported Google product. It is a reference implementation. There is absolutely NO WARRANTY provided for using this code. The code is Apache Licensed and CAN BE fully modified, white labeled, and disassembled by your team.

This code generated (see starthinker/scripts for possible source):
  - **Command**: "python starthinker_ui/manage.py colab"
  - **Command**: "python starthinker/tools/colab.py [JSON RECIPE]"



#1. Install Dependencies
First install the libraries needed to execute recipes, this only needs to be done once, then click play.


In [ ]:
!pip install git+https://github.com/google/starthinker


#2. Set Configuration

This code is required to initialize the project. Fill in required fields and press play.

1. If the recipe uses a Google Cloud Project:
  - Set the configuration **project** value to the project identifier from [these instructions](https://github.com/google/starthinker/blob/master/tutorials/cloud_project.md).

1. If the recipe has **auth** set to **user**:
  - If you have user credentials:
    - Set the configuration **user** value to your user credentials JSON.
  - If you DO NOT have user credentials:
    - Set the configuration **client** value to [downloaded client credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_client_installed.md).

1. If the recipe has **auth** set to **service**:
  - Set the configuration **service** value to [downloaded service credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_service.md).



In [ ]:
from starthinker.util.configuration import Configuration


CONFIG = Configuration(
  project="",
  client={},
  service={},
  user="/content/user.json",
  verbose=True
)



#3. Enter DV360 User Audit Recipe Parameters
 1. DV360 only permits SERVICE accounts to access the user list API endpoint, be sure to provide and permission one.
 1. Wait for **BigQuery->->->DV_...** to be created.
 1. Wait for **BigQuery->->->Barnacle_...** to be created, then copy and connect the following data sources.
 1. Join the [StarThinker Assets Group](https://groups.google.com/d/forum/starthinker-assets) to access the following assets
 1. Copy [Barnacle DV Report](https://datastudio.google.com/c/u/0/reporting/9f6b9e62-43ec-4027-849a-287e9c1911bd).
 1. Click Edit->Resource->Manage added data sources, then edit each connection to connect to your new tables above.
 1. Or give these intructions to the client.
Modify the values below for your use case, can be done multiple times, then click play.


In [ ]:
FIELDS = {
  'auth_read':'user',  # Credentials used for writing data.
  'auth_write':'service',  # Credentials used for writing data.
  'partner':'',  # Partner ID to run user audit on.
  'recipe_slug':'',  # Name of Google BigQuery dataset to create.
}

print("Parameters Set To: %s" % FIELDS)


#4. Execute DV360 User Audit
This does NOT need to be modified unless you are changing the recipe, click play.


In [ ]:
from starthinker.util.configuration import execute
from starthinker.util.recipe import json_set_fields

TASKS = [
  {
    'dataset':{
      'auth':{'field':{'name':'auth_write','kind':'authentication','order':1,'default':'service','description':'Credentials used for writing data.'}},
      'dataset':{'field':{'name':'recipe_slug','kind':'string','order':4,'default':'','description':'Name of Google BigQuery dataset to create.'}}
    }
  },
  {
    'google_api':{
      'auth':{'field':{'name':'auth_read','kind':'authentication','order':0,'default':'user','description':'Credentials used for writing data.'}},
      'api':'doubleclickbidmanager',
      'version':'v1.1',
      'function':'queries.listqueries',
      'alias':'list',
      'results':{
        'bigquery':{
          'auth':{'field':{'name':'auth_write','kind':'authentication','order':1,'default':'service','description':'Credentials used for writing data.'}},
          'dataset':{'field':{'name':'recipe_slug','kind':'string','order':4,'default':'','description':'Name of Google BigQuery dataset to create.'}},
          'table':'DV_Reports'
        }
      }
    }
  },
  {
    'google_api':{
      'auth':{'field':{'name':'auth_read','kind':'authentication','order':0,'default':'user','description':'Credentials used for writing data.'}},
      'api':'displayvideo',
      'version':'v1',
      'function':'partners.list',
      'kwargs':{
        'fields':'partners.displayName,partners.partnerId,nextPageToken'
      },
      'results':{
        'bigquery':{
          'auth':{'field':{'name':'auth_write','kind':'authentication','order':1,'default':'service','description':'Credentials used for writing data.'}},
          'dataset':{'field':{'name':'recipe_slug','kind':'string','order':4,'default':'','description':'Name of Google BigQuery dataset to create.'}},
          'table':'DV_Partners'
        }
      }
    }
  },
  {
    'google_api':{
      'auth':{'field':{'name':'auth_read','kind':'authentication','order':0,'default':'user','description':'Credentials used for writing data.'}},
      'api':'displayvideo',
      'version':'v1',
      'function':'advertisers.list',
      'kwargs':{
        'partnerId':{'field':{'name':'partner','kind':'integer','order':2,'default':'','description':'Partner ID to run user audit on.'}},
        'fields':'advertisers.displayName,advertisers.advertiserId,nextPageToken'
      },
      'results':{
        'bigquery':{
          'auth':{'field':{'name':'auth_write','kind':'authentication','order':1,'default':'service','description':'Credentials used for writing data.'}},
          'dataset':{'field':{'name':'recipe_slug','kind':'string','order':4,'default':'','description':'Name of Google BigQuery dataset to create.'}},
          'table':'DV_Advertisers'
        }
      }
    }
  },
  {
    'google_api':{
      'auth':'service',
      'api':'displayvideo',
      'version':'v1',
      'function':'users.list',
      'kwargs':{
      },
      'results':{
        'bigquery':{
          'auth':{'field':{'name':'auth_write','kind':'authentication','order':1,'default':'service','description':'Credentials used for writing data.'}},
          'dataset':{'field':{'name':'recipe_slug','kind':'string','order':4,'default':'','description':'Name of Google BigQuery dataset to create.'}},
          'table':'DV_Users'
        }
      }
    }
  },
  {
    'bigquery':{
      'auth':{'field':{'name':'auth_write','kind':'authentication','order':1,'default':'service','description':'Credentials used for writing data.'}},
      'from':{
        'query':"SELECT           U.userId,           U.name,           U.email,           U.displayName,           REGEXP_EXTRACT(U.email, r'@(.+)') AS Domain,           IF (ENDS_WITH(U.email, '.gserviceaccount.com'), 'Service', 'User') AS Authentication,           IF((Select COUNT(advertiserId) from UNNEST(U.assignedUserRoles)) = 0, 'Partner', 'Advertiser') AS Scope,           STRUCT(             AUR.partnerId,             P.displayName AS partnerName,             AUR.userRole,             AUR.advertiserId,             A.displayName AS advertiserName,             AUR.assignedUserRoleId           ) AS assignedUserRoles,           FROM `{dataset}.DV_Users` AS U,           UNNEST(assignedUserRoles) AS AUR           LEFT JOIN `{dataset}.DV_Partners` AS P           ON AUR.partnerId=P.partnerId           LEFT JOIN `{dataset}.DV_Advertisers` AS A           ON AUR.advertiserId=A.advertiserId         ",
        'parameters':{
          'dataset':{'field':{'name':'recipe_slug','kind':'string','order':4,'default':'','description':'Name of Google BigQuery dataset to create.'}}
        },
        'legacy':False
      },
      'to':{
        'dataset':{'field':{'name':'recipe_slug','kind':'string','order':4,'default':'','description':'Name of Google BigQuery dataset to create.'}},
        'view':'Barnacle_User_Roles'
      }
    }
  },
  {
    'bigquery':{
      'auth':{'field':{'name':'auth_write','kind':'authentication','order':1,'default':'service','description':'Credentials used for writing data.'}},
      'from':{
        'query':"SELECT           R.*,           P.displayName AS partnerName,           A.displayName AS advertiserName,           FROM (           SELECT             queryId,             (SELECT CAST(value AS INT64) FROM UNNEST(R.params.filters) WHERE type = 'FILTER_PARTNER' LIMIT 1) AS partnerId,             (SELECT CAST(value AS INT64) FROM UNNEST(R.params.filters) WHERE type = 'FILTER_ADVERTISER' LIMIT 1) AS advertiserId,             R.schedule.frequency,             R.params.metrics,             R.params.type,             R.metadata.dataRange,             R.metadata.sendNotification,             DATE(TIMESTAMP_MILLIS(R.metadata.latestReportRunTimeMS)) AS latestReportRunTime,           FROM `{dataset}.DV_Reports` AS R) AS R           LEFT JOIN `{dataset}.DV_Partners` AS P           ON R.partnerId=P.partnerId           LEFT JOIN `{dataset}.DV_Advertisers` AS A           ON R.advertiserId=A.advertiserId         ",
        'parameters':{
          'dataset':{'field':{'name':'recipe_slug','kind':'string','order':4,'default':'','description':'Name of Google BigQuery dataset to create.'}}
        },
        'legacy':False
      },
      'to':{
        'dataset':{'field':{'name':'recipe_slug','kind':'string','order':4,'default':'','description':'Name of Google BigQuery dataset to create.'}},
        'table':'Barnacle_Reports'
      }
    }
  }
]

json_set_fields(TASKS, FIELDS)

execute(CONFIG, TASKS, force=True)
